In [1]:
import psycopg2
import pandas as pd
import numpy as np
from sqlalchemy import create_engine


In [4]:
user_data = pd.read_sql(
    """SELECT * FROM public.user_data""",
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

user_data

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads
...,...,...,...,...,...,...,...,...
163200,168548,0,36,Russia,Kaliningrad,4,Android,organic
163201,168549,0,18,Russia,Tula,2,Android,organic
163202,168550,1,41,Russia,Yekaterinburg,4,Android,organic
163203,168551,0,38,Russia,Moscow,3,iOS,organic


In [5]:
feed_data = pd.read_sql(
    """SELECT  *
                FROM public.feed_data feed_data
                WHERE feed_data.timestamp >= to_timestamp('26.12.21', 'DD.MM.YY' ) 
                  AND feed_data.action = 'view'""",
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

feed_data

,timestamp,user_id,post_id,action,target
0,2021-12-27 21:32:56,164976,4685,view,0
1,2021-12-27 21:35:07,164976,6542,view,0
2,2021-12-27 21:35:18,164976,3040,view,0
3,2021-12-27 21:36:41,164976,5597,view,0
4,2021-12-27 21:38:27,164976,779,view,1
...,...,...,...,...,...
3091891,2021-12-29 14:42:37,81478,4893,view,0
3091892,2021-12-29 14:43:51,81478,2635,view,0
3091893,2021-12-29 14:45:28,81478,5015,view,0
3091894,2021-12-29 14:45:44,81478,154,view,0


In [6]:
post_data = pd.read_sql(
    """SELECT * FROM public.post_text_df """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

post_data

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie
7019,7316,I give this movie 2 stars purely because of it...,movie
7020,7317,I cant believe this film was allowed to be mad...,movie
7021,7318,The version I saw of this film was the Blockbu...,movie


In [7]:
def text_column_processing(X):    
        ## Метод TF-IDF
        from sklearn.feature_extraction.text import TfidfVectorizer
        tf_idf = TfidfVectorizer()
        text_tf_idf = tf_idf.fit_transform(X['text'])

        ## Метод главных компонет сверху TF-IDF
        from sklearn.decomposition import PCA
        pca = PCA(n_components = 3)
        PCA_post_text_dataset = pca.fit_transform(text_tf_idf.toarray())
        PCA_post_text_dataset = pd.DataFrame(PCA_post_text_dataset, 
                                             columns=['PCA_text_1', 'PCA_text_2', 'PCA_text_3'], 
                                             index=X['post_id'])
        return PCA_post_text_dataset

In [8]:
PCA_post_text_dataset = text_column_processing(post_data[['post_id', 'text']].drop_duplicates(subset='post_id'))

In [9]:
PCA_post_text_dataset

,PCA_text_1,PCA_text_2,PCA_text_3
post_id,,,
1,-0.118109,-0.202089,-0.091933
2,-0.142662,-0.224652,-0.000819
3,-0.104280,-0.152657,-0.088866
4,-0.001531,-0.127432,-0.054625
5,0.055888,-0.104884,-0.004086
...,...,...,...
7315,-0.076983,0.210305,0.128411
7316,-0.086584,0.131223,0.072674
7317,0.017890,0.105220,-0.016941


In [10]:
post_PCA = pd.merge(
    PCA_post_text_dataset,
    post_data,
    on='post_id',
    how='left'
)

post_PCA

,post_id,PCA_text_1,PCA_text_2,PCA_text_3,text,topic
0,1,-0.118109,-0.202089,-0.091933,UK economy facing major risks\n\nThe UK manufa...,business
1,2,-0.142662,-0.224652,-0.000819,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,-0.104280,-0.152657,-0.088866,Asian quake hits European shares\n\nShares in ...,business
3,4,-0.001531,-0.127432,-0.054625,India power shares jump on debut\n\nShares in ...,business
4,5,0.055888,-0.104884,-0.004086,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...,...,...,...
7018,7315,-0.076983,0.210305,0.128411,"OK, I would not normally watch a Farrelly brot...",movie
7019,7316,-0.086584,0.131223,0.072674,I give this movie 2 stars purely because of it...,movie
7020,7317,0.017890,0.105220,-0.016941,I cant believe this film was allowed to be mad...,movie
7021,7318,-0.135336,0.119797,0.012214,The version I saw of this film was the Blockbu...,movie


### Контентный подход

In [11]:
#Удаляем столбец action тк в нем всего одно значение
feed_data.drop('action', axis=1, inplace = True)
feed_data

,timestamp,user_id,post_id,target
0,2021-12-27 21:32:56,164976,4685,0
1,2021-12-27 21:35:07,164976,6542,0
2,2021-12-27 21:35:18,164976,3040,0
3,2021-12-27 21:36:41,164976,5597,0
4,2021-12-27 21:38:27,164976,779,1
...,...,...,...,...
3091891,2021-12-29 14:42:37,81478,4893,0
3091892,2021-12-29 14:43:51,81478,2635,0
3091893,2021-12-29 14:45:28,81478,5015,0
3091894,2021-12-29 14:45:44,81478,154,0


In [12]:
feed_data['timestamp'] = pd.to_datetime(feed_data['timestamp'])
feed_data['hour_of_action'] = feed_data['timestamp'].dt.hour

In [13]:
feed_data

,timestamp,user_id,post_id,target,hour_of_action
0,2021-12-27 21:32:56,164976,4685,0,21
1,2021-12-27 21:35:07,164976,6542,0,21
2,2021-12-27 21:35:18,164976,3040,0,21
3,2021-12-27 21:36:41,164976,5597,0,21
4,2021-12-27 21:38:27,164976,779,1,21
...,...,...,...,...,...
3091891,2021-12-29 14:42:37,81478,4893,0,14
3091892,2021-12-29 14:43:51,81478,2635,0,14
3091893,2021-12-29 14:45:28,81478,5015,0,14
3091894,2021-12-29 14:45:44,81478,154,0,14


In [14]:
df = pd.merge(
    feed_data,
    user_data,
    on='user_id',
    how='left'
)

df

,timestamp,user_id,post_id,target,hour_of_action,gender,age,country,city,exp_group,os,source
0,2021-12-27 21:32:56,164976,4685,0,21,1,47,Russia,Novozybkov,0,iOS,organic
1,2021-12-27 21:35:07,164976,6542,0,21,1,47,Russia,Novozybkov,0,iOS,organic
2,2021-12-27 21:35:18,164976,3040,0,21,1,47,Russia,Novozybkov,0,iOS,organic
3,2021-12-27 21:36:41,164976,5597,0,21,1,47,Russia,Novozybkov,0,iOS,organic
4,2021-12-27 21:38:27,164976,779,1,21,1,47,Russia,Novozybkov,0,iOS,organic
...,...,...,...,...,...,...,...,...,...,...,...,...
3091891,2021-12-29 14:42:37,81478,4893,0,14,0,21,Russia,Lyudinovo,2,Android,ads
3091892,2021-12-29 14:43:51,81478,2635,0,14,0,21,Russia,Lyudinovo,2,Android,ads
3091893,2021-12-29 14:45:28,81478,5015,0,14,0,21,Russia,Lyudinovo,2,Android,ads
3091894,2021-12-29 14:45:44,81478,154,0,14,0,21,Russia,Lyudinovo,2,Android,ads


In [15]:
df = pd.merge(
    df,
    post_PCA,
    on='post_id',
    how='left'
)

df.head()

,timestamp,user_id,post_id,target,hour_of_action,gender,age,country,city,exp_group,os,source,PCA_text_1,PCA_text_2,PCA_text_3,text,topic
0,2021-12-27 21:32:56,164976,4685,0,21,1,47,Russia,Novozybkov,0,iOS,organic,-0.076086,0.151342,0.014963,I cannot see why anyone would make such a movi...,movie
1,2021-12-27 21:35:07,164976,6542,0,21,1,47,Russia,Novozybkov,0,iOS,organic,-0.189369,0.022530,0.004020,I know that there are some purists out there w...,movie
2,2021-12-27 21:35:18,164976,3040,0,21,1,47,Russia,Novozybkov,0,iOS,organic,0.252496,-0.004799,-0.008169,Pierre Dufour joins YourTV to discuss the @Cor...,covid
3,2021-12-27 21:36:41,164976,5597,0,21,1,47,Russia,Novozybkov,0,iOS,organic,-0.182676,-0.009720,-0.087353,Hollywood North is an euphemism from the movie...,movie
4,2021-12-27 21:38:27,164976,779,1,21,1,47,Russia,Novozybkov,0,iOS,organic,-0.045372,-0.089701,-0.042046,Branson show flops on US screens\n\nEntreprene...,entertainment


In [16]:
df

,timestamp,user_id,post_id,target,hour_of_action,gender,age,country,city,exp_group,os,source,PCA_text_1,PCA_text_2,PCA_text_3,text,topic
0,2021-12-27 21:32:56,164976,4685,0,21,1,47,Russia,Novozybkov,0,iOS,organic,-0.076086,0.151342,0.014963,I cannot see why anyone would make such a movi...,movie
1,2021-12-27 21:35:07,164976,6542,0,21,1,47,Russia,Novozybkov,0,iOS,organic,-0.189369,0.022530,0.004020,I know that there are some purists out there w...,movie
2,2021-12-27 21:35:18,164976,3040,0,21,1,47,Russia,Novozybkov,0,iOS,organic,0.252496,-0.004799,-0.008169,Pierre Dufour joins YourTV to discuss the @Cor...,covid
3,2021-12-27 21:36:41,164976,5597,0,21,1,47,Russia,Novozybkov,0,iOS,organic,-0.182676,-0.009720,-0.087353,Hollywood North is an euphemism from the movie...,movie
4,2021-12-27 21:38:27,164976,779,1,21,1,47,Russia,Novozybkov,0,iOS,organic,-0.045372,-0.089701,-0.042046,Branson show flops on US screens\n\nEntreprene...,entertainment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3091891,2021-12-29 14:42:37,81478,4893,0,14,0,21,Russia,Lyudinovo,2,Android,ads,-0.189456,0.108917,-0.030894,I heard about this movie when watching VH1s 10...,movie
3091892,2021-12-29 14:43:51,81478,2635,0,14,0,21,Russia,Lyudinovo,2,Android,ads,0.228477,-0.008134,0.016219,Please please now is the time to put politics ...,covid
3091893,2021-12-29 14:45:28,81478,5015,0,14,0,21,Russia,Lyudinovo,2,Android,ads,-0.129867,0.064373,0.002171,"Although I like Kurt Vonnegut, Im not particul...",movie
3091894,2021-12-29 14:45:44,81478,154,0,14,0,21,Russia,Lyudinovo,2,Android,ads,-0.066543,-0.174295,-0.037492,Economy strong in election year\n\nUK business...,business


In [17]:
#While we delete column "Text", we'll deal with it later
df.drop('text', axis=1, inplace = True)
df

,timestamp,user_id,post_id,target,hour_of_action,gender,age,country,city,exp_group,os,source,PCA_text_1,PCA_text_2,PCA_text_3,topic
0,2021-12-27 21:32:56,164976,4685,0,21,1,47,Russia,Novozybkov,0,iOS,organic,-0.076086,0.151342,0.014963,movie
1,2021-12-27 21:35:07,164976,6542,0,21,1,47,Russia,Novozybkov,0,iOS,organic,-0.189369,0.022530,0.004020,movie
2,2021-12-27 21:35:18,164976,3040,0,21,1,47,Russia,Novozybkov,0,iOS,organic,0.252496,-0.004799,-0.008169,covid
3,2021-12-27 21:36:41,164976,5597,0,21,1,47,Russia,Novozybkov,0,iOS,organic,-0.182676,-0.009720,-0.087353,movie
4,2021-12-27 21:38:27,164976,779,1,21,1,47,Russia,Novozybkov,0,iOS,organic,-0.045372,-0.089701,-0.042046,entertainment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3091891,2021-12-29 14:42:37,81478,4893,0,14,0,21,Russia,Lyudinovo,2,Android,ads,-0.189456,0.108917,-0.030894,movie
3091892,2021-12-29 14:43:51,81478,2635,0,14,0,21,Russia,Lyudinovo,2,Android,ads,0.228477,-0.008134,0.016219,covid
3091893,2021-12-29 14:45:28,81478,5015,0,14,0,21,Russia,Lyudinovo,2,Android,ads,-0.129867,0.064373,0.002171,movie
3091894,2021-12-29 14:45:44,81478,154,0,14,0,21,Russia,Lyudinovo,2,Android,ads,-0.066543,-0.174295,-0.037492,business


In [18]:
categorical_columns = df.loc[:,df.dtypes==np.object_].columns
categorical_columns

Index(['country', 'city', 'os', 'source', 'topic'], dtype='object')

In [19]:
categorical_columns_user = user_data.loc[:,user_data.dtypes==np.object_].columns
categorical_columns_user

Index(['country', 'city', 'os', 'source'], dtype='object')

In [20]:
df.describe(include='object')

,country,city,os,source,topic
count,3091896,3091896,3091896,3091896,3091896
unique,11,3341,2,2,7
top,Russia,Moscow,Android,ads,movie
freq,2711678,416512,2009608,1926118,1144766


In [21]:
df['target'].value_counts()

target
0    2667879
1     424017
Name: count, dtype: int64

In [22]:
### Функция для multiclass encoding'а
### Пока откладываем этот метод
def encode_multiclass_target(df, category, target='target'):
    df = df[[category, target]].copy()
    
    target_dummies = pd.get_dummies(df[target], 
                                    prefix=category, 
                                    drop_first=True)
    
    df = pd.concat((df, target_dummies), axis=1)
    
    for tg in target_dummies.columns:
        
        df[tg] = df.groupby(category)[tg].transform("mean")
    
    return df.drop(category, axis=1)


In [23]:
for col in categorical_columns:
    if df[col].nunique() < 8:
        one_hot = pd.get_dummies(df[col], prefix=col, drop_first=True)     
        df = pd.concat((df.drop(col, axis=1), one_hot), axis=1)
    else:
        mean_target = encode_multiclass_target(df, col).drop("target", axis=1)
        df = pd.concat((df.drop(col, axis=1), mean_target), axis=1)

In [24]:
df['user_id'].nunique()

80952

In [25]:
df

,timestamp,user_id,post_id,target,hour_of_action,gender,age,exp_group,PCA_text_1,PCA_text_2,...,country_1,city_1,os_iOS,source_organic,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,2021-12-27 21:32:56,164976,4685,0,21,1,47,0,-0.076086,0.151342,...,0.129955,0.154275,True,True,False,False,True,False,False,False
1,2021-12-27 21:35:07,164976,6542,0,21,1,47,0,-0.189369,0.022530,...,0.129955,0.154275,True,True,False,False,True,False,False,False
2,2021-12-27 21:35:18,164976,3040,0,21,1,47,0,0.252496,-0.004799,...,0.129955,0.154275,True,True,True,False,False,False,False,False
3,2021-12-27 21:36:41,164976,5597,0,21,1,47,0,-0.182676,-0.009720,...,0.129955,0.154275,True,True,False,False,True,False,False,False
4,2021-12-27 21:38:27,164976,779,1,21,1,47,0,-0.045372,-0.089701,...,0.129955,0.154275,True,True,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3091891,2021-12-29 14:42:37,81478,4893,0,14,0,21,2,-0.189456,0.108917,...,0.129955,0.114047,False,False,False,False,True,False,False,False
3091892,2021-12-29 14:43:51,81478,2635,0,14,0,21,2,0.228477,-0.008134,...,0.129955,0.114047,False,False,True,False,False,False,False,False
3091893,2021-12-29 14:45:28,81478,5015,0,14,0,21,2,-0.129867,0.064373,...,0.129955,0.114047,False,False,False,False,True,False,False,False
3091894,2021-12-29 14:45:44,81478,154,0,14,0,21,2,-0.066543,-0.174295,...,0.129955,0.114047,False,False,False,False,False,False,False,False


In [26]:
user_count_views = df.groupby('user_id').size()
user_means = df.groupby('user_id')['target'].sum()

In [27]:
df['userViews'] = df['user_id'].map(user_count_views)
df['userMeans'] = df['user_id'].map(user_means) 

In [28]:
### Разделим выборку на train-test

df_witout_time = df.drop(['timestamp'], axis=1)

In [29]:
from sklearn.model_selection import train_test_split
X = df.drop('target', axis=1)
y = df['target']

In [30]:
#Выборку делим на train-test по времени
#Train все что было за 3 часа до, в тест последние 3 часа действий
d = df['timestamp'].max() - pd.Timedelta(hours=3) 
X_test, X_train = X[X.timestamp >= d], X[X.timestamp < d]
y_test, y_train = y[y.index.isin(X_test.index)], y[y.index.isin(X_train.index)]
X_train = X_train.drop(['user_id','timestamp'], axis=1)

In [116]:
X_test_without_id = X_test.drop(['user_id','timestamp'], axis=1)

In [117]:
X_test_without_id

,post_id,hour_of_action,gender,age,exp_group,PCA_text_1,PCA_text_2,PCA_text_3,country_1,city_1,os_iOS,source_organic,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech,userViews,userMeans
1744,671,21,1,33,4,-0.053187,-0.083534,0.067097,0.129955,0.170186,False,False,False,True,False,False,False,False,29,6
1745,59,21,1,33,4,-0.004492,-0.123222,-0.046583,0.129955,0.170186,False,False,False,False,False,False,False,False,29,6
1746,6441,21,1,33,4,-0.155520,0.113802,0.073984,0.129955,0.170186,False,False,False,False,True,False,False,False,29,6
1747,2528,21,1,33,4,0.320484,-0.005197,-0.010041,0.129955,0.170186,False,False,True,False,False,False,False,False,29,6
1748,6165,21,1,33,4,-0.061661,0.049173,-0.022976,0.129955,0.170186,False,False,False,False,True,False,False,False,29,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3091012,2770,22,1,43,0,0.214111,-0.008100,0.050154,0.129955,0.161501,False,True,True,False,False,False,False,False,58,9
3091013,1264,22,1,43,0,-0.198764,-0.207595,0.009276,0.129955,0.161501,False,True,False,False,False,True,False,False,58,9
3091014,1468,22,1,43,0,-0.041592,-0.111914,0.036032,0.129955,0.161501,False,True,False,False,False,False,True,False,58,9
3091015,867,22,1,43,0,-0.044761,-0.069976,-0.113312,0.129955,0.161501,False,True,False,True,False,False,False,False,58,9


In [118]:
y_train.value_counts()

target
0    2568225
1     405903
Name: count, dtype: int64

In [119]:
from catboost import CatBoostClassifier, Pool
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
pipe = Pipeline([
                 ("scaler", StandardScaler()),  
                 ("cat_boost_estimator",  CatBoostClassifier()),
                 ])
pipe.fit(X_train, y_train)

Learning rate set to 0.313371
0:	learn: 0.5061510	total: 265ms	remaining: 4m 25s
1:	learn: 0.4314278	total: 442ms	remaining: 3m 40s
2:	learn: 0.4002509	total: 619ms	remaining: 3m 25s
3:	learn: 0.3851048	total: 781ms	remaining: 3m 14s
4:	learn: 0.3783883	total: 949ms	remaining: 3m 8s
5:	learn: 0.3748582	total: 1.08s	remaining: 2m 59s
6:	learn: 0.3718205	total: 1.25s	remaining: 2m 57s
7:	learn: 0.3709592	total: 1.4s	remaining: 2m 53s
8:	learn: 0.3691116	total: 1.55s	remaining: 2m 50s
9:	learn: 0.3686953	total: 1.68s	remaining: 2m 46s
10:	learn: 0.3684800	total: 1.82s	remaining: 2m 43s
11:	learn: 0.3676339	total: 1.97s	remaining: 2m 41s
12:	learn: 0.3672954	total: 2.11s	remaining: 2m 40s
13:	learn: 0.3670275	total: 2.25s	remaining: 2m 38s
14:	learn: 0.3669201	total: 2.39s	remaining: 2m 36s
15:	learn: 0.3668019	total: 2.52s	remaining: 2m 35s
16:	learn: 0.3664138	total: 2.66s	remaining: 2m 33s
17:	learn: 0.3661583	total: 2.79s	remaining: 2m 32s
18:	learn: 0.3660806	total: 2.94s	remaining: 2

158:	learn: 0.3614076	total: 23.7s	remaining: 2m 5s
159:	learn: 0.3613929	total: 23.8s	remaining: 2m 5s
160:	learn: 0.3613833	total: 24s	remaining: 2m 4s
161:	learn: 0.3613708	total: 24.1s	remaining: 2m 4s
162:	learn: 0.3613449	total: 24.3s	remaining: 2m 4s
163:	learn: 0.3613335	total: 24.4s	remaining: 2m 4s
164:	learn: 0.3613168	total: 24.6s	remaining: 2m 4s
165:	learn: 0.3612890	total: 24.7s	remaining: 2m 4s
166:	learn: 0.3612566	total: 24.8s	remaining: 2m 3s
167:	learn: 0.3612255	total: 25s	remaining: 2m 3s
168:	learn: 0.3612044	total: 25.1s	remaining: 2m 3s
169:	learn: 0.3611976	total: 25.3s	remaining: 2m 3s
170:	learn: 0.3611741	total: 25.4s	remaining: 2m 3s
171:	learn: 0.3611510	total: 25.6s	remaining: 2m 3s
172:	learn: 0.3611360	total: 25.7s	remaining: 2m 3s
173:	learn: 0.3611140	total: 25.9s	remaining: 2m 2s
174:	learn: 0.3610894	total: 26s	remaining: 2m 2s
175:	learn: 0.3610655	total: 26.2s	remaining: 2m 2s
176:	learn: 0.3610523	total: 26.3s	remaining: 2m 2s
177:	learn: 0.3610

316:	learn: 0.3592238	total: 47s	remaining: 1m 41s
317:	learn: 0.3592056	total: 47.1s	remaining: 1m 41s
318:	learn: 0.3591907	total: 47.3s	remaining: 1m 40s
319:	learn: 0.3591786	total: 47.4s	remaining: 1m 40s
320:	learn: 0.3591722	total: 47.6s	remaining: 1m 40s
321:	learn: 0.3591602	total: 47.7s	remaining: 1m 40s
322:	learn: 0.3591477	total: 47.8s	remaining: 1m 40s
323:	learn: 0.3591397	total: 48s	remaining: 1m 40s
324:	learn: 0.3591255	total: 48.1s	remaining: 1m 39s
325:	learn: 0.3591061	total: 48.3s	remaining: 1m 39s
326:	learn: 0.3590859	total: 48.4s	remaining: 1m 39s
327:	learn: 0.3590661	total: 48.6s	remaining: 1m 39s
328:	learn: 0.3590455	total: 48.7s	remaining: 1m 39s
329:	learn: 0.3590334	total: 48.9s	remaining: 1m 39s
330:	learn: 0.3590243	total: 49s	remaining: 1m 39s
331:	learn: 0.3590178	total: 49.2s	remaining: 1m 38s
332:	learn: 0.3590030	total: 49.3s	remaining: 1m 38s
333:	learn: 0.3589881	total: 49.5s	remaining: 1m 38s
334:	learn: 0.3589802	total: 49.6s	remaining: 1m 38s

473:	learn: 0.3575068	total: 1m 11s	remaining: 1m 19s
474:	learn: 0.3574872	total: 1m 11s	remaining: 1m 19s
475:	learn: 0.3574819	total: 1m 11s	remaining: 1m 19s
476:	learn: 0.3574758	total: 1m 12s	remaining: 1m 19s
477:	learn: 0.3574668	total: 1m 12s	remaining: 1m 18s
478:	learn: 0.3574630	total: 1m 12s	remaining: 1m 18s
479:	learn: 0.3574530	total: 1m 12s	remaining: 1m 18s
480:	learn: 0.3574439	total: 1m 12s	remaining: 1m 18s
481:	learn: 0.3574369	total: 1m 12s	remaining: 1m 18s
482:	learn: 0.3574039	total: 1m 12s	remaining: 1m 18s
483:	learn: 0.3573870	total: 1m 13s	remaining: 1m 17s
484:	learn: 0.3573786	total: 1m 13s	remaining: 1m 17s
485:	learn: 0.3573716	total: 1m 13s	remaining: 1m 17s
486:	learn: 0.3573635	total: 1m 13s	remaining: 1m 17s
487:	learn: 0.3573566	total: 1m 13s	remaining: 1m 17s
488:	learn: 0.3573508	total: 1m 13s	remaining: 1m 17s
489:	learn: 0.3573441	total: 1m 14s	remaining: 1m 17s
490:	learn: 0.3573384	total: 1m 14s	remaining: 1m 17s
491:	learn: 0.3573288	total:

627:	learn: 0.3561642	total: 1m 36s	remaining: 57.3s
628:	learn: 0.3561594	total: 1m 36s	remaining: 57.2s
629:	learn: 0.3561543	total: 1m 37s	remaining: 57s
630:	learn: 0.3561470	total: 1m 37s	remaining: 56.9s
631:	learn: 0.3561319	total: 1m 37s	remaining: 56.8s
632:	learn: 0.3561253	total: 1m 37s	remaining: 56.6s
633:	learn: 0.3561095	total: 1m 37s	remaining: 56.5s
634:	learn: 0.3561035	total: 1m 38s	remaining: 56.4s
635:	learn: 0.3560956	total: 1m 38s	remaining: 56.2s
636:	learn: 0.3560885	total: 1m 38s	remaining: 56.1s
637:	learn: 0.3560782	total: 1m 38s	remaining: 55.9s
638:	learn: 0.3560719	total: 1m 38s	remaining: 55.8s
639:	learn: 0.3560660	total: 1m 38s	remaining: 55.6s
640:	learn: 0.3560602	total: 1m 39s	remaining: 55.5s
641:	learn: 0.3560538	total: 1m 39s	remaining: 55.4s
642:	learn: 0.3560406	total: 1m 39s	remaining: 55.2s
643:	learn: 0.3560253	total: 1m 39s	remaining: 55.1s
644:	learn: 0.3560183	total: 1m 39s	remaining: 54.9s
645:	learn: 0.3560092	total: 1m 39s	remaining: 5

783:	learn: 0.3549830	total: 2m 3s	remaining: 34s
784:	learn: 0.3549755	total: 2m 3s	remaining: 33.8s
785:	learn: 0.3549644	total: 2m 3s	remaining: 33.6s
786:	learn: 0.3549617	total: 2m 3s	remaining: 33.5s
787:	learn: 0.3549564	total: 2m 3s	remaining: 33.3s
788:	learn: 0.3549488	total: 2m 4s	remaining: 33.2s
789:	learn: 0.3549372	total: 2m 4s	remaining: 33s
790:	learn: 0.3549307	total: 2m 4s	remaining: 32.9s
791:	learn: 0.3549237	total: 2m 4s	remaining: 32.7s
792:	learn: 0.3549177	total: 2m 4s	remaining: 32.6s
793:	learn: 0.3549128	total: 2m 4s	remaining: 32.4s
794:	learn: 0.3549056	total: 2m 5s	remaining: 32.2s
795:	learn: 0.3548993	total: 2m 5s	remaining: 32.1s
796:	learn: 0.3548938	total: 2m 5s	remaining: 31.9s
797:	learn: 0.3548859	total: 2m 5s	remaining: 31.8s
798:	learn: 0.3548787	total: 2m 5s	remaining: 31.6s
799:	learn: 0.3548694	total: 2m 5s	remaining: 31.5s
800:	learn: 0.3548632	total: 2m 5s	remaining: 31.3s
801:	learn: 0.3548574	total: 2m 6s	remaining: 31.1s
802:	learn: 0.35

939:	learn: 0.3539079	total: 2m 29s	remaining: 9.53s
940:	learn: 0.3539020	total: 2m 29s	remaining: 9.37s
941:	learn: 0.3538914	total: 2m 29s	remaining: 9.21s
942:	learn: 0.3538832	total: 2m 29s	remaining: 9.06s
943:	learn: 0.3538768	total: 2m 30s	remaining: 8.9s
944:	learn: 0.3538699	total: 2m 30s	remaining: 8.74s
945:	learn: 0.3538637	total: 2m 30s	remaining: 8.58s
946:	learn: 0.3538562	total: 2m 30s	remaining: 8.42s
947:	learn: 0.3538513	total: 2m 30s	remaining: 8.27s
948:	learn: 0.3538464	total: 2m 30s	remaining: 8.11s
949:	learn: 0.3538413	total: 2m 31s	remaining: 7.95s
950:	learn: 0.3538358	total: 2m 31s	remaining: 7.79s
951:	learn: 0.3538303	total: 2m 31s	remaining: 7.63s
952:	learn: 0.3538228	total: 2m 31s	remaining: 7.47s
953:	learn: 0.3538180	total: 2m 31s	remaining: 7.31s
954:	learn: 0.3538006	total: 2m 31s	remaining: 7.15s
955:	learn: 0.3537949	total: 2m 32s	remaining: 7s
956:	learn: 0.3537888	total: 2m 32s	remaining: 6.84s
957:	learn: 0.3537836	total: 2m 32s	remaining: 6.6

Pipeline(steps=[('scaler', StandardScaler()),
                ('cat_boost_estimator',
                 <catboost.core.CatBoostClassifier object at 0x2e4a56f10>)])

In [128]:
X_test_without_id.columns

Index(['gender', 'age', 'exp_group', 'post_id', 'hour_of_action',
       'minute_of_action', 'PCA_text_1', 'PCA_text_2', 'PCA_text_3',
       'userViews', 'userMeans', 'country_1.0', 'city_1.0', 'os_iOS',
       'source_organic', 'topic_covid', 'topic_entertainment', 'topic_movie',
       'topic_politics', 'topic_sport', 'topic_tech'],
      dtype='object')

In [120]:
from sklearn.metrics import classification_report
y_pred = pipe.predict(X_test_without_id)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.85      1.00      0.92     99654
           1       0.39      0.01      0.02     18114

    accuracy                           0.85    117768
   macro avg       0.62      0.50      0.47    117768
weighted avg       0.78      0.85      0.78    117768



In [121]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

y_pred = pipe.predict(X_test_without_id)
y_pred_proba = pipe.predict_proba(X_test_without_id)[:, 1] # вероятности класса 1

print(f'balanced_accuracy = {balanced_accuracy_score(y_test, y_pred)}')
print(f'accuracy = {accuracy_score(y_test, y_pred)}')
print(f'f1_score = {f1_score(y_test, y_pred)}')

balanced_accuracy = 0.5040229472211679
accuracy = 0.8451616737993343
f1_score = 0.02198980960042907


In [123]:
### При расчете исключаю пользователей, которые не ставили хотя бы одного лайка
X_test_ = X_test[['user_id','post_id']]

X_test_['pred'] = y_pred
X_test_['pred_proba'] = y_pred_proba
X_test_['target'] = y_test

hitrate_list = []
user_without_likes = []
unique_user_count = 0

for user in X_test_['user_id'].unique():
    # Выбираем строки с конкретным пользователем
    part = X_test_[X_test_['user_id']==user]
    
    # Вычисляем, есть ли у пользователя хотя бы 1 лайк
    temp = [1 if part['target'].sum()==0 else 0]
    
    # если у пользователя нет хотя бы 1-ого лайка,
    # то не включаем его в расчет метрики
    if np.sum(temp)==1:
        user_without_likes.append(temp)
        continue
    else:
        # сортируем посты строки по вероятности класса 1 (лайка)
        # выбираем 5 строк с наибольшей вероятностью лайка
        part = part.sort_values(by='pred_proba', ascending=False).head(5)
        # если правильно предсказан хотя бы 1 пост,
        # то ставим 1, иначе 0
        hitrate = [1 if (part['target']).sum() > 0 else 0]   
        
        hitrate_list.append(hitrate)
        unique_user_count += 1
    
print(f"Среднее hitrate по пользователям из теста: {np.round(np.sum(hitrate_list)/unique_user_count, 2)}")
print(f"Доля пользователей без лайков: {np.round(np.sum(user_without_likes)/X_test_['user_id'].unique().shape[0], 2)}")

/var/folders/w1/w_59zgln13d9rzr047_dbkww0000gn/T/ipykernel_4396/3729158146.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_['pred'] = y_pred
/var/folders/w1/w_59zgln13d9rzr047_dbkww0000gn/T/ipykernel_4396/3729158146.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_['pred_proba'] = y_pred_proba
/var/folders/w1/w_59zgln13d9rzr047_dbkww0000gn/T/ipykernel_4396/3729158146.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

Среднее hitrate по пользователям из теста: 0.7
Доля пользователей без лайков: 0.14


In [124]:
import pickle
filename = 'Models/catboost_model_2.pkl'
pickle.dump(pipe, open(filename, 'wb'))

In [80]:
import os
from catboost import CatBoostClassifier
import pickle

def get_model_path(path: str) -> str:
    if os.environ.get("IS_LMS") == "1":  # проверяем где выполняется код в лмс, или локально. Немного магии
        MODEL_PATH = '/workdir/user_input/model'
    else:
        MODEL_PATH = path
    return MODEL_PATH

def load_models():
    model_path = get_model_path("./catboost_model.pkl")
    loaded_model = pickle.load(open(model_path, 'rb'))
    loaded_model.predict(X_train)# пример как можно загружать модели

In [81]:
load_models()

In [80]:

one_hot = pd.get_dummies(post_data['topic'], prefix='topic', drop_first=True)     
df_post = pd.concat((post_data.drop('topic', axis=1), one_hot), axis=1)

In [81]:
def text_column_processing(X):    
        ## Метод TF-IDF
        from sklearn.feature_extraction.text import TfidfVectorizer
        tf_idf = TfidfVectorizer()
        text_tf_idf = tf_idf.fit_transform(X['text'])

        ## Метод главных компонет сверху TF-IDF
        from sklearn.decomposition import PCA
        pca = PCA(n_components = 3)
        PCA_post_text_dataset = pca.fit_transform(text_tf_idf.toarray())
        PCA_post_text_dataset = pd.DataFrame(PCA_post_text_dataset, 
                                             columns=['PCA_text_1', 'PCA_text_2', 'PCA_text_3'], 
                                             index=X['post_id'])
        return PCA_post_text_dataset

In [82]:
PCA_post_text_dataset = text_column_processing(df_post[['post_id', 'text']].drop_duplicates(subset='post_id'))
df_post = pd.merge(
    PCA_post_text_dataset,
    df_post,
    on='post_id',
    how='left'
).drop(['text'],axis=1)

df_post

,post_id,PCA_text_1,PCA_text_2,PCA_text_3,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,1,-0.118109,-0.202089,-0.091981,False,False,False,False,False,False
1,2,-0.142662,-0.224652,-0.000745,False,False,False,False,False,False
2,3,-0.104280,-0.152657,-0.088959,False,False,False,False,False,False
3,4,-0.001531,-0.127431,-0.054667,False,False,False,False,False,False
4,5,0.055888,-0.104884,-0.004144,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
7018,7315,-0.076983,0.210306,0.128334,False,False,True,False,False,False
7019,7316,-0.086584,0.131223,0.072740,False,False,True,False,False,False
7020,7317,0.017890,0.105220,-0.016867,False,False,True,False,False,False
7021,7318,-0.135336,0.119798,0.011991,False,False,True,False,False,False


In [83]:
df_post.to_sql('rusind899_lesson_22_2', if_exists = 'replace', con=engine) # записываем таблицу

23

In [127]:
df_post

,post_id,PCA_text_1,PCA_text_2,PCA_text_3,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,1,-0.118109,-0.202089,-0.091981,False,False,False,False,False,False
1,2,-0.142662,-0.224652,-0.000745,False,False,False,False,False,False
2,3,-0.104280,-0.152657,-0.088959,False,False,False,False,False,False
3,4,-0.001531,-0.127431,-0.054667,False,False,False,False,False,False
4,5,0.055888,-0.104884,-0.004144,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
7018,7315,-0.076983,0.210306,0.128334,False,False,True,False,False,False
7019,7316,-0.086584,0.131223,0.072740,False,False,True,False,False,False
7020,7317,0.017890,0.105220,-0.016867,False,False,True,False,False,False
7021,7318,-0.135336,0.119798,0.011991,False,False,True,False,False,False


### Test predict

In [129]:
user_tab = dff[dff['user_id']==200][['user_id', 'gender', 'age', 'exp_group','country_1.0', 'city_1.0', 'os_iOS',
       'source_organic','userViews','userMeans']]
df_post['user_id']=200
user_tab['timestamp'] = '2021-12-29 21:14:34'
user_tab['timestamp'] = pd.to_datetime(user_tab['timestamp'])
user_tab['hour_of_action'] = user_tab['timestamp'].dt.hour

In [130]:
user_tab

,user_id,gender,age,exp_group,country_1.0,city_1.0,os_iOS,source_organic,userViews,userMeans,timestamp,hour_of_action
0,200,1.0,34.0,3.0,0.130122,0.174278,0.0,0.0,51.0,6.0,2021-12-29 21:14:34,21


In [131]:
df_test = pd.merge(
    df_post,
    user_tab,
    on='user_id',
    how='left'
)

df_test

,post_id,PCA_text_1,PCA_text_2,PCA_text_3,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech,...,age,exp_group,country_1.0,city_1.0,os_iOS,source_organic,userViews,userMeans,timestamp,hour_of_action
0,1,-0.118109,-0.202089,-0.091981,False,False,False,False,False,False,...,34.0,3.0,0.130122,0.174278,0.0,0.0,51.0,6.0,2021-12-29 21:14:34,21
1,2,-0.142662,-0.224652,-0.000745,False,False,False,False,False,False,...,34.0,3.0,0.130122,0.174278,0.0,0.0,51.0,6.0,2021-12-29 21:14:34,21
2,3,-0.104280,-0.152657,-0.088959,False,False,False,False,False,False,...,34.0,3.0,0.130122,0.174278,0.0,0.0,51.0,6.0,2021-12-29 21:14:34,21
3,4,-0.001531,-0.127431,-0.054667,False,False,False,False,False,False,...,34.0,3.0,0.130122,0.174278,0.0,0.0,51.0,6.0,2021-12-29 21:14:34,21
4,5,0.055888,-0.104884,-0.004144,False,False,False,False,False,False,...,34.0,3.0,0.130122,0.174278,0.0,0.0,51.0,6.0,2021-12-29 21:14:34,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,-0.076983,0.210306,0.128334,False,False,True,False,False,False,...,34.0,3.0,0.130122,0.174278,0.0,0.0,51.0,6.0,2021-12-29 21:14:34,21
7019,7316,-0.086584,0.131223,0.072740,False,False,True,False,False,False,...,34.0,3.0,0.130122,0.174278,0.0,0.0,51.0,6.0,2021-12-29 21:14:34,21
7020,7317,0.017890,0.105220,-0.016867,False,False,True,False,False,False,...,34.0,3.0,0.130122,0.174278,0.0,0.0,51.0,6.0,2021-12-29 21:14:34,21
7021,7318,-0.135336,0.119798,0.011991,False,False,True,False,False,False,...,34.0,3.0,0.130122,0.174278,0.0,0.0,51.0,6.0,2021-12-29 21:14:34,21


In [139]:
df_test_without = df_test.drop(['user_id','timestamp'],axis = 1)
df_test_without = df_test_without.rename(columns={"country_1.0": "country_1", "city_1.0": "city_1"})

In [140]:
df_test_without.columns

Index(['post_id', 'PCA_text_1', 'PCA_text_2', 'PCA_text_3', 'topic_covid',
       'topic_entertainment', 'topic_movie', 'topic_politics', 'topic_sport',
       'topic_tech', 'gender', 'age', 'exp_group', 'country_1', 'city_1',
       'os_iOS', 'source_organic', 'userViews', 'userMeans', 'hour_of_action'],
      dtype='object')

In [142]:
pred_test_ = pipe.predict(df_test_without[['post_id', 'hour_of_action', 'gender', 'age', 'exp_group', 'PCA_text_1',
       'PCA_text_2', 'PCA_text_3', 'country_1', 'city_1', 'os_iOS',
       'source_organic', 'topic_covid', 'topic_entertainment', 'topic_movie',
       'topic_politics', 'topic_sport', 'topic_tech', 'userViews',
       'userMeans']])


In [144]:
X = df_test.drop(['user_id', 'timestamp'], axis = 1).rename(columns={"country_1.0": "country_1", "city_1.0": "city_1"})[['post_id', 'hour_of_action', 'gender', 'age', 'exp_group', 'PCA_text_1',
       'PCA_text_2', 'PCA_text_3', 'country_1', 'city_1', 'os_iOS',
       'source_organic', 'topic_covid', 'topic_entertainment', 'topic_movie',
       'topic_politics', 'topic_sport', 'topic_tech', 'userViews',
       'userMeans']]
preds = pipe.predict(X)
y_pred_proba = pipe.predict_proba(X)[:, 1]
X['pred'] = preds
X['pred_proba'] = y_pred_proba
part = X.sort_values(by='pred_proba', ascending=False).head(5)

In [147]:
answ=post_data[post_data['post_id'].isin([358, 331, 330, 330, 330])]

In [148]:
answ
for index, row in answ.iterrows():
    print(row.to_dict())

{'post_id': 358, 'text': 'Deutsche Boerse set to woo LSE\n\nBosses of Deutsche Boerse and the London Stock Exchange are to meet amid talk that a takeover bid for the LSE will be raised to £1.5bn ($2.9bn).\n\nLast month, the German exchange tabled a 530 pence-per-share offer for LSE, valuing it at £1.3bn. Paris-based Euronext, owner of Liffe in London, has also said it is interested in bidding for LSE. Euronext is due to hold talks with LSE this week and it is reported to be ready to raise £1.4bn to fund a bid. Euronext chief Jean-Francois Theodore is scheduled to meet his LSE counterpart Clara Furse on Friday. Deutsche Boerse chief Werner Seifert is meeting Ms Furse on Thursday, in the third meeting between the two exchanges since the bid approach in December.\n\nThe LSE rejected Deutsche Boerses proposed £1.3bn offer in December, saying it undervalued the business.\n\nBut it agreed to leave the door open for talks to find out whether a significantly-improved proposal would be in the i

In [143]:
part['post_id'].to_list()

[358, 331, 330, 330, 330]